In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [2]:
simple_transform=transforms.Compose([transforms.ToTensor()])
exp_dataset=datasets.CIFAR10('./data',train=True,download=True,transform=simple_transform)

Files already downloaded and verified


In [3]:
train_loader = torch.utils.data.DataLoader(exp_dataset, batch_size=len(exp_dataset), shuffle=False)
data = next(iter(train_loader))

# Calculate mean and std per channel
mean = data[0].mean(dim=(0, 2, 3))  # Calculate mean across all images and height, width dimensions
std = data[0].std(dim=(0, 2, 3))    # Calculate std across all images and height, width dimensions

print("Mean per channel:", mean)
print("Std per channel:", std)

Mean per channel: tensor([0.4914, 0.4822, 0.4465])
Std per channel: tensor([0.2470, 0.2435, 0.2616])


In [4]:
import cv2
import torchvision

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


class Cifar10SearchDataset(torchvision.datasets.CIFAR10):
    def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label
train_transform=A.Compose([
                            A.Normalize((0.4914, 0.4822, 0.4465),(0.2470, 0.2435, 0.2616)),
                           A.HorizontalFlip(),
                           A.ShiftScaleRotate (shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, interpolation=1, border_mode=4, value=None, mask_value=None, shift_limit_x=None, shift_limit_y=None, rotate_method='largest_box', always_apply=False, p=0.5),
                           A.CoarseDropout(max_holes = 1, max_height=16, max_width=16, min_holes = 1, min_height=16, min_width=16, fill_value=(0.4914, 0.4822, 0.4465), mask_fill_value=None),
                            ToTensorV2()
                           
                                   ])
test_transform=A.Compose([A.Normalize((0.4914, 0.4822, 0.4465),(0.2470, 0.2435, 0.2616)),
                          ToTensorV2()
                                   ])
train_data=Cifar10SearchDataset('./data',train=True,download=True,transform=train_transform)
test_data=Cifar10SearchDataset('./data',train=False,download=True,transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

CUDA Available? False


In [6]:
from model import Net
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             864
       BatchNorm2d-2           [-1, 32, 30, 30]              64
              ReLU-3           [-1, 32, 30, 30]               0
            Conv2d-4           [-1, 64, 15, 15]          18,432
       BatchNorm2d-5           [-1, 64, 15, 15]             128
              ReLU-6           [-1, 64, 15, 15]               0
            Conv2d-7           [-1, 32, 15, 15]           2,048
            Conv2d-8           [-1, 32, 15, 15]           9,216
       BatchNorm2d-9           [-1, 32, 15, 15]              64
             ReLU-10           [-1, 32, 15, 15]               0
           Conv2d-11           [-1, 32, 15, 15]           9,216
      BatchNorm2d-12           [-1, 32, 15, 15]              64
             ReLU-13           [-1, 32, 15, 15]               0
           Conv2d-14             [-

In [9]:
from train import train,test
device = torch.device("mps")
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
train_losses = []
test_losses = []
train_acc = []
test_acc = []
for epoch in range(1, 41):
    print(epoch)
    train_loss,train_accuracy=train(model, device, train_loader, optimizer, epoch)
    test_loss,test_accuracy=test(model, device, test_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_acc.append(train_accuracy)
    test_acc.append(test_accuracy)

1


loss=1.594364047050476 batch_id=781 Accuracy=35.93: 100%|█| 782/782 [00:32<00:00



Test set: Average loss: 1.4632, Accuracy: 4604/10000 (46%)

2


loss=1.4038478136062622 batch_id=781 Accuracy=47.22: 100%|█| 782/782 [00:36<00:0



Test set: Average loss: 1.3943, Accuracy: 5193/10000 (52%)

3


...


Test set: Average loss: 0.5226, Accuracy: 8214/10000 (82%)

39


loss=0.8540305495262146 batch_id=781 Accuracy=76.21: 100%|█| 782/782 [00:26<00:0



Test set: Average loss: 0.5145, Accuracy: 8258/10000 (83%)

40


loss=0.6189147233963013 batch_id=781 Accuracy=76.03: 100%|█| 782/782 [00:25<00:0



Test set: Average loss: 0.5001, Accuracy: 8317/10000 (83%)

